<a href="https://colab.research.google.com/github/ArthurSampaio13/Previsao-precos-apartamento-com-GCP/blob/main/Previsao_apartamentos_RJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Construindo um Modelo de Regressão Linear para Previsão de Preços de
## Airbnbs no Rio de Janeiro

Arthur Lopes Filgueira Sampaio

### Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np

### Abrindo o arquivo listings.csv do Airbnb Rio de Janeiro

In [ ]:
df = pd.read_csv("listings.csv", index_col="id")

## Analisando linhas e colunas

In [ ]:
print(f'Quantidade de linhas: {df.shape[0]}')
print(f'Quantidade de colunas: {df.shape[1]}')

Quantidade de linhas: 31964
Quantidade de colunas: 74


### Analisando valores da tabela

In [ ]:
df.describe()

,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,3.196400e+04,3.196400e+04,31961.000000,31961.000000,0.0,31964.000000,31964.000000,31964.000000,0.0,24617.000000,...,23668.000000,23667.000000,23665.000000,23667.00000,0.0,31964.000000,31964.000000,31964.000000,31964.000000,23824.000000
mean,2.023092e+13,1.663077e+08,16.112450,27.273020,NaN,-22.967574,-43.251361,4.041265,NaN,1.783117,...,4.881593,4.866827,4.861961,4.69660,NaN,8.839319,8.010825,0.707139,0.111719,0.974190
std,0.000000e+00,1.684315e+08,89.082309,134.207472,NaN,0.035182,0.100595,2.370380,NaN,1.067153,...,0.321587,0.346049,0.322802,0.44839,NaN,23.515196,23.328498,1.735520,0.918342,1.127053
min,2.023092e+13,1.671000e+03,1.000000,1.000000,NaN,-23.073276,-43.723009,1.000000,NaN,1.000000,...,1.000000,1.000000,1.000000,1.00000,NaN,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.023092e+13,2.258631e+07,1.000000,1.000000,NaN,-22.984820,-43.310047,2.000000,NaN,1.000000,...,4.890000,4.870000,4.850000,4.61000,NaN,1.000000,1.000000,0.000000,0.000000,0.170000
50%,2.023092e+13,8.693849e+07,2.000000,3.000000,NaN,-22.972860,-43.195670,4.000000,NaN,2.000000,...,5.000000,5.000000,4.980000,4.81000,NaN,2.000000,1.000000,0.000000,0.000000,0.570000
75%,2.023092e+13,3.004090e+08,5.000000,7.000000,NaN,-22.956165,-43.185946,5.000000,NaN,2.000000,...,5.000000,5.000000,5.000000,5.00000,NaN,4.000000,3.000000,1.000000,0.000000,1.380000
max,2.023092e+13,5.379850e+08,1311.000000,1803.000000,NaN,-22.749690,-43.104400,16.000000,NaN,26.000000,...,5.000000,5.000000,5.000000,5.00000,NaN,163.000000,159.000000,18.000000,15.000000,14.780000


### Analisando o tipo das colunas

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31964 entries, 231497 to 985555107088259155
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_url                                   31964 non-null  object 
 1   scrape_id                                     31964 non-null  int64  
 2   last_scraped                                  31964 non-null  object 
 3   source                                        31964 non-null  object 
 4   name                                          31964 non-null  object 
 5   description                                   31305 non-null  object 
 6   neighborhood_overview                         16739 non-null  object 
 7   picture_url                                   31964 non-null  object 
 8   host_id                                       31964 non-null  int64  
 9   host_url                                   

### Analisando e ordenando valores nulos e valores únicos

In [ ]:
df_info = pd.DataFrame({'valores_nulos':np.round(df.isnull().mean(), 2),
              'valores_unicos': df.nunique()})
df_info = df_info.sort_values('valores_nulos', ascending=False)
df_info

,valores_nulos,valores_unicos
neighbourhood_group_cleansed,1.0,0
license,1.0,0
bathrooms,1.0,0
calendar_updated,1.0,0
host_about,0.5,7803
...,...,...
minimum_maximum_nights,0.0,175
maximum_maximum_nights,0.0,175
minimum_nights_avg_ntm,0.0,240
maximum_nights_avg_ntm,0.0,730


# Tratamento dos dados

### Colunas que considerei importante

In [ ]:
colunas_desejadas = ['host_since', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_listings_count',
                     'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
                     'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights',
                     'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                     'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 'availability_60',
                     'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
                     'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                     'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable',
                     'reviews_per_month']

### Colunas que considerei mais importante

In [ ]:
variaveis_com_pesos_altos = [
    'host_since',
    'host_is_superhost',
    'host_total_listings_count',
    'host_listings_count',
    'property_type',
    'room_type',
    'accommodates',
    'bathrooms_text',
    'bedrooms',
    'beds',
    'amenities',
    'number_of_reviews',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'price',
    'availability_30',
    'availability_60',
    'availability_90',
    'availability_365'
]

### Análise destas colunas

In [ ]:
df[variaveis_com_pesos_altos].describe()

,host_total_listings_count,host_listings_count,accommodates,bedrooms,beds,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,availability_30,availability_60,availability_90,availability_365
count,31961.000000,31961.000000,31964.000000,24617.000000,31789.000000,31964.000000,31964.000000,31964.000000,23824.000000,23667.000000,23668.000000,23668.000000,23667.000000,23665.000000,23667.00000,23824.000000,31964.000000,31964.000000,31964.000000,31964.000000
mean,27.273020,16.112450,4.041265,1.783117,2.502690,19.938274,7.017426,0.584845,4.749393,4.800967,4.736847,4.881593,4.866827,4.861961,4.69660,0.974190,16.451133,33.953416,54.673883,188.491960
std,134.207472,89.082309,2.370380,1.067153,2.202994,40.676059,11.436773,1.167655,0.558515,0.400921,0.451490,0.321587,0.346049,0.322802,0.44839,1.127053,11.989915,23.314947,33.898050,133.012558
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.010000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,4.720000,4.760000,4.670000,4.890000,4.870000,4.850000,4.61000,0.170000,3.000000,10.000000,24.000000,72.000000
50%,3.000000,2.000000,4.000000,2.000000,2.000000,4.000000,2.000000,0.000000,4.910000,4.930000,4.890000,5.000000,5.000000,4.980000,4.81000,0.570000,19.000000,39.000000,65.000000,174.000000
75%,7.000000,5.000000,5.000000,2.000000,3.000000,20.000000,9.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,1.380000,29.000000,58.000000,88.000000,335.000000
max,1803.000000,1311.000000,16.000000,26.000000,91.000000,618.000000,133.000000,13.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,14.780000,30.000000,60.000000,90.000000,365.000000


In [ ]:
df_estudo = df[variaveis_com_pesos_altos].isnull().sum()
df_estudo

host_since                        3
host_is_superhost               656
host_total_listings_count         3
host_listings_count               3
property_type                     0
room_type                         0
accommodates                      0
bathrooms_text                   28
bedrooms                       7347
beds                            175
amenities                         0
number_of_reviews                 0
number_of_reviews_ltm             0
number_of_reviews_l30d            0
review_scores_rating           8140
review_scores_accuracy         8297
review_scores_cleanliness      8296
review_scores_checkin          8296
review_scores_communication    8297
review_scores_location         8299
review_scores_value            8297
reviews_per_month              8140
price                             0
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
dtype: int64

### Substituindo o t por 1 e o f por 0 (Para facilitar a análise do GCP)

In [ ]:
df['host_is_superhost'] = df['host_is_superhost'].replace({'t': '1', 'f': '0'})

### Preenchendo valores nulos em super host (Como os valores não são True, logo são falsos (Princípio do terceiro excluído))

In [ ]:
df['host_is_superhost'].fillna(0, inplace=True)

### Preenchendo valores nulos de review (Notei que os reviews_per_month estavam zerados, logo não tem como haver métricas para o cálculo)

In [ ]:
df['review_scores_rating'].fillna(0, inplace=True)

df['review_scores_accuracy'].fillna(0, inplace=True)

df['review_scores_cleanliness'].fillna(0, inplace=True)

df['review_scores_checkin'].fillna(0, inplace=True)

df['review_scores_communication'].fillna(0, inplace=True)

df['review_scores_location'].fillna(0, inplace=True)

df['review_scores_value'].fillna(0, inplace=True)

df['reviews_per_month'].fillna(0, inplace=True)

### Removando os dados vazios de banheiros_text (28 ao todo)

In [ ]:
df.dropna(subset=['bathrooms_text'], inplace=True)

### Removendo o '$' da coluna price e convertendo o tipo (com isso surgiram 4380 valores nulos em price)

In [ ]:
df['price'] = df['price'].str.replace('$','')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

<ipython-input-17-60cee67c6195>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$','')


### Removendo os outliers





In [ ]:
df = df[df['price'] < 700]

In [ ]:
df = df[df['host_total_listings_count'] < 50]

In [ ]:
df = df[df['host_listings_count'] < 50]

In [ ]:
df = df[df['accommodates'] < 10]

In [ ]:
df = df[df['bedrooms'] < 10]

In [ ]:
df = df[df['beds'] < 10]

In [ ]:
df = df[df['number_of_reviews'] < 50]

In [ ]:
df = df[df['number_of_reviews_ltm'] < 50]

In [ ]:
df = df[df['number_of_reviews_l30d'] < 50]

In [ ]:
df = df[df['reviews_per_month'] < 150]

### Substituindo valores nulos pela média

In [ ]:
media_bedrooms = df['bedrooms'].mean()
df['bedrooms'].fillna(media_bedrooms, inplace=True)

media_beds = df['beds'].mean()
df['beds'].fillna(media_beds, inplace=True)

media_price = df['price'].mean()
df['price'].fillna(media_price, inplace=True)

media_host_total = df['host_total_listings_count'].mean()
df['host_total_listings_count'].fillna(media_host_total, inplace=True)

media_listing = df['host_listings_count'].mean()
df['host_listings_count'].fillna(media_listing, inplace=True)

### Conferindo os novos resultados

In [ ]:
df.describe()

,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,2.287100e+04,2.287100e+04,22871.000000,22871.000000,0.0,22871.000000,22871.000000,22871.000000,0.0,22871.000000,...,22871.000000,22871.000000,22871.000000,22871.000000,0.0,22871.000000,22871.000000,22871.000000,22871.000000,22871.000000
mean,2.023092e+13,1.853158e+08,3.857603,5.244658,NaN,-22.965170,-43.254367,3.579686,NaN,1.664741,...,3.547001,3.538810,3.524153,3.413665,NaN,3.386166,2.504001,0.746535,0.128197,0.590033
std,0.000000e+00,1.740367e+08,5.850839,7.891855,NaN,0.036606,0.102713,1.701520,NaN,0.676233,...,2.197518,2.195398,2.185442,2.135747,NaN,5.263837,4.965484,1.801540,0.951379,0.936716
min,2.023092e+13,1.671000e+03,1.000000,1.000000,NaN,-23.073276,-43.723009,1.000000,NaN,1.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.023092e+13,3.093908e+07,1.000000,1.000000,NaN,-22.984318,-43.322946,2.000000,NaN,1.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,1.000000,0.000000,0.000000,0.000000
50%,2.023092e+13,1.104963e+08,2.000000,2.000000,NaN,-22.970360,-43.195467,4.000000,NaN,1.782727,...,4.940000,4.930000,4.890000,4.670000,NaN,1.000000,1.000000,0.000000,0.000000,0.200000
75%,2.023092e+13,3.352654e+08,4.000000,5.000000,NaN,-22.948045,-43.185405,4.000000,NaN,2.000000,...,5.000000,5.000000,5.000000,4.970000,NaN,3.000000,2.000000,1.000000,0.000000,0.780000
max,2.023092e+13,5.379850e+08,43.000000,48.000000,NaN,-22.749950,-43.104400,9.000000,NaN,7.000000,...,5.000000,5.000000,5.000000,5.000000,NaN,42.000000,42.000000,18.000000,15.000000,12.470000


### Apagando colunas que estão completamente vazias

In [ ]:
df.drop('neighbourhood_group_cleansed', axis=1, inplace=True)
df.drop('license', axis=1, inplace=True)
df.drop('bathrooms', axis=1, inplace=True)
df.drop('calendar_updated', axis=1, inplace=True)

### Imprimindo a média geral de preços

In [ ]:
print(f'A média geral de preços é: R${df.price.mean():.2f}')

A média geral de preços é: R$293.08


# Outras aplicações para a base de dados seria:

### Maiores médias gerais de preço por bairro:

In [ ]:
médias_por_região = df.groupby('neighbourhood_cleansed')['price'].mean().sort_values(ascending=False)
médias_por_região[:5]

neighbourhood_cleansed
Cascadura      415.000000
Leblon         387.164624
Jacaré         381.666667
Joá            368.307134
São Conrado    360.982736
Name: price, dtype: float64

### Os bairros que são mais comentandos nas reviews:

In [ ]:
bairros_comentados = df.groupby(['neighbourhood_cleansed']).number_of_reviews.mean().sort_values(ascending=False)
print(bairros_comentados[:5])

neighbourhood_cleansed
Deodoro                35.000000
Magalhães Bastos       33.000000
Senador Vasconcelos    18.166667
Pitangueiras           16.000000
Benfica                15.500000
Name: number_of_reviews, dtype: float64


### Host mais comentado nas reviews:

In [ ]:
host = df.groupby(['host_name']).number_of_reviews.mean().sort_values(ascending=False)
host[:5]

host_name
Dircélia       49.0
Vana           49.0
Jolni          49.0
Weverson       49.0
Carmen Dora    49.0
Name: number_of_reviews, dtype: float64

### Valor total de todos os alugueis:



In [ ]:
print(f'O valor total seria: R${round(sum(df.price), 2)}')

O valor total seria: R$6703102.08


### Exportando o arquivo tratado

In [ ]:
df.to_csv('data_set_limpo.csv')